In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import csv
from collections import defaultdict
import json
import sqlite3
from sqlite3 import Error
from queue import Queue 

In [ ]:
# terms_dict = defaultdict(list)
terms_list = []
appDictionary = {}
relatedSearchTerms = {}

In [ ]:
def commaSeparated(term):
    result = "";
    if len(term.split(' ')) > 1:
        count = len(term.split(' '));
        i = 0;
        for word in term.split(' '):
            if i > 0:
                result = result + "+";
            result = result + word;
            i = i + 1;
        return result;
    else:
        return term;

In [ ]:
#Class to store app details in object format
class AppDetails:
    def __init__(self, title, description, stars, appID, imageSource, developerName):
        self.title = title
        self.description = description
        self.stars = stars
        self.appID = appID
        self.imageSource = imageSource
        self.developerName = developerName
        
    def toString(application):
        return application.title + ", " + application.appID + ", " + application.description + ", " + application.stars + ", " + application.imageSource + ", " + application.developerName

In [ ]:
#DB DRIVERS

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
            
            
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
sql_create_appDetailsMainTable_table = """ CREATE TABLE IF NOT EXISTS appDetailsMainTable (
                                        key text NOT NULL,
                                        appIDs text NOT NULL,
                                        relatedSearchTerms text,
                                        source text NOT NULL
                                    ); """

sql_create_appDetails_table = """ CREATE TABLE IF NOT EXISTS appDetails (
                                        appID text NOT NULL,
                                        title text NOT NULL,
                                        description text,
                                        stars text,
                                        imageSource text,
                                        developerName text,
                                        source text
                                    ); """

sql_create_relatedSearchTerms_table = """ CREATE TABLE IF NOT EXISTS relatedSearchTerms (
                                        keyTerm text PRIMARY KEY,
                                        terms text NOT NULL,
                                        source text
                                    ); """

def databaseStartUp():
    # Create connection
    conn = create_connection(r"malavida_database.db")

    # Create table
    if conn is not None:
        create_table(conn, sql_create_appDetailsMainTable_table)
        create_table(conn, sql_create_appDetails_table)
        create_table(conn, sql_create_relatedSearchTerms_table)
        print("Database created!")
    else:
        print("Error! cannot create the database connection.")
    
    return conn;
        
        
def insertIntoAppDetailsMainTable(conn, task):
 
    sql = ''' INSERT INTO appDetailsMainTable(key,appIDs,relatedSearchTerms,source)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

def insertIntoAppDetails(conn, task):
 
    sql = ''' INSERT INTO appDetails(appID, title, description, stars, imageSource, developerName, source)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid
     
conn = databaseStartUp()

In [ ]:
with open('android_terms.csv','rt')as f:
  data = csv.reader(f)
  lineNumber = 0;
  for row in data:
        if lineNumber >= 1:
            key = row[2];
            terms = row[3];
            i = 0;
            for term in terms.split('"'):
                if i%2 == 0:
                    i = i+1;
                    continue
                else:
                    i = i+1;
                    terms_list.append(term)
        else:
            lineNumber = lineNumber + 1;

In [ ]:
finalTermsList = []

In [ ]:
finalTermsList.clear()
for term in terms_list:
    result = commaSeparated(term)
    finalTermsList.append(result)

In [ ]:
len(finalTermsList)

432331

In [ ]:
# WRITTEN FOR TESTING PURPOSE PLEASE IGNORE 

payload = {'q': 'mspy', 't': 'app'}
r = requests.get('https://apk.support/search', params=payload);
print(r.url)
print(r.status_code)

In [ ]:
# TEST
soup = BeautifulSoup(r.text, 'html.parser')
app_table = soup.find_all("h3")
appNamesList = []
for app in app_table:
    appName = app.get_text()
    appNamesList.append(appName)
appDictionary["mspy"] = appNamesList


# TEST Cleanup List
for term in finalTermsList:
    if(term == ""):
        finalTermsList.remove(term)
singleWordSet = set();
for term in finalTermsList:
    if(term.find("+") == -1 and term != ""):
        singleWordSet.add(term)

In [ ]:
q = Queue();
wordSet = set();
for term in finalTermsList:
    if(term != ""):
        wordSet.add(term)
print(len(wordSet))
for word in wordSet:
    q.put(word)
print(q.qsize())

In [ ]:
# FOR TEST PURPOSE
singleWordSet = {'cheater', 'cheaters', 'cheating', 'track husband', 'keylogger', 'keyloggers'}

In [ ]:
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word, 't': 'app'};
    r = requests.get('https://apk.support/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "it_column"})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    for name in names_table:
        # Developer Information
        developerPart = name.find_all("div", attrs={"class": "ss_tg"})
        developerPart = developerPart[0].find_all("a")
        developerTag = developerPart[0]['href']
        developerTag = developerTag[10:]
        developerName = developerPart[0].get_text()
        information = name.find_all("a")
        # Title
        titleTag = information[0].find_all("h3")
        title = titleTag[0].get_text()
#         print("Title is " + title)
        # Description
        descriptionTag = information[0].find_all("p")
        description = descriptionTag[0].get_text()
#         print("Description is " + description)
        # Stars
        starsTag = information[0].find_all("div", attrs = {"class" : "stars"})
        starsSpan = starsTag[0].find_all("span")
        stars = starsSpan[0]['title']
        starCount = stars[stars.rindex(' ')+1:]
#         print("Stars is " + starCount)
        # AppID
        appID = information[0]['href']
        appID = appID[4 : ]
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
#         print("AppID:- " + appID)
        # Image Source Link
        imageTag = information[0].find_all("div", attrs={"class" : "seo_img"})
        imageTag = imageTag[0].find_all("img")
        imageSource = imageTag[0]['data-original']
#         print(" ImageSrc:- " + imageSource)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, stars, imageSource, developerName, 'apk.support');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        
    # Suggestion Addition
    suggestionList = soup.find_all("div", attrs={"class": "suggest"})
    suggestionList = suggestionList[0].find_all("li")
    suggestions = []
    suggestionsString = "";
    i = 0
    for suggestion in suggestionList:
        suggestionName = suggestion.get_text()
        if (i != 0):
            suggestionsString = suggestionsString + ","
        suggestionsString = suggestionsString + suggestionName
        i = 1
        suggestions.append(suggestionName)
        modifiedSuggestionName = commaSeparated(suggestionName)
        if(modifiedSuggestionName not in wordSet):
#             print("Added " + modifiedSuggestionName)
            wordSet.add(modifiedSuggestionName)
            q.put(modifiedSuggestionName)
#         else:
#             print("Not Adding " + modifiedSuggestionName)
    
    #Insert Into Main Table
    taskMainTable = (word, appIDList, suggestionsString, 'apk.support');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
    
    
#     appDictionary[word] = finalList
#     relatedSearchTerms[word] = suggestions
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    print("Completed " + word)

In [ ]:
#https://apkgk.com/
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'keyword': word}
    r = requests.get('https://apkgk.com/search/', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appId = soup.find_all("ul", attrs={"class":'topic-wrap'})
    if(len(appId) == 0):
        print("Skipping " + word)
        continue
    appId = appId[0].find_all("a")
    names_table = soup.find_all('div', attrs={"class":'topic-bg'})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    counter = 0
    for name in names_table:
        appName = name.find_all("div", attrs={"class": "topic-tip-name"})
        appDesc = name.find_all("div", attrs={"class": "topic-tip-description"})
        appSrcMain  = name.find_all("div", attrs={"class": "c-lz-load"})
        imageTag = appSrcMain[0].find_all("img")
        appLink = imageTag[0]['data-src']
        #for img in appSrcMain:
        print(appId[counter]['href'])
        print(appLink)
        print(appName[0].get_text())
        print(appDesc[0].get_text())
        
        appID = appId[counter]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        imageSource = appLink
        description = appDesc[0].get_text()
        title = appName[0].get_text()
        starCount = "NULL"
        developerName = "NULL"
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkgk.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkgk.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
        


In [ ]:
#https://www.malavida.com
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    word = word.replace('+','-')
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    #payload = {'/':word}
    r = requests.get('https://www.malavida.com/en/s/'+word);
    soup = BeautifulSoup(r.text, 'html.parser');
    appDetails = soup.find_all("section", attrs={"class":'app-list'})
    appList = soup.find_all("section", attrs={"class":'app-download'})
    counter = 0
    finalList = []
    appIDList = ""
    first = 0
    notFound = soup.find_all("section", attrs={"class":'not-found'})
    if(len(notFound)>0):
        print("Skipping " + word)
        continue
    print(len(appList))
    for app in appList:
        appSrc = app.find_all("div", attrs={"class":"title"})
        appDesc = app.find_all("p")
        imageSource = app.find_all('img')
        imageSource = imageSource[0]['src']
        appLink = appSrc[0].find_all("a")
        appID = appLink[0]['href']
        title = appLink[0].get_text()
        description = appDesc[0].get_text()
        counter = counter+1
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        starCount = "NULL"
        developerName = "NULL"
        print(title +", "+description+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'malavida.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'malavida.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
    
    

In [ ]:
# Map check Code
for app in appDictionary:
    appList = appDictionary[app]
    print("Entries for " + app)
    for application in appList:
        print(application.toString())
        print("------------------------------->>>")

In [ ]:
# Saving dict to file
with open('apksupport.json', 'w') as f:
    json.dump(appDictionary, f)

with open('relatedSearchTerms_apksupport.json', 'w') as f:
    json.dump(relatedSearchTerms, f)

In [ ]:
SQLite3
dataset package python

upper table -> key and set of app ids
lower table -> app id is primary key and we write to this.

In [ ]:
conn.close()

In [ ]:
# FOR APKCOMBO.COM (ERROR: CLOUDFLARE BLOCKING)
qTrial = Queue()
qTrial.put("cheating+wife")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(5)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkcombo.com/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    print(soup.prettify())
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "grid-apps mb-10"})
    

In [ ]:
# FOR APKPURE.COM
qTrial = Queue()
qTrial.put("Starting hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    checkMore = 1
    firstCheck = 1
    interval = 0
    appIDList = ""
    first = 0
    while(checkMore):
        if(firstCheck == 0):
            interval = interval + 15
        payload = {'q': word,  't': 'app', 'begin': interval};
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        r = requests.get('https://apkpure.com/search-page', params=payload_str);
        soup = BeautifulSoup(r.text, 'html.parser')
        #print(soup.prettify())
        names_table = soup.find_all("dl", attrs={"class": "search-dl"})
        #print(names_table)
        if(len(names_table) == 0):
            print("Skipping " + word)
            break
        for name in names_table:
            dtPart = name.find_all("dt");
            ddPart = name.find_all("dd");
            aPart = dtPart[0].find_all("a");
            appID = aPart[0]['href']
            imagePart = aPart[0].find_all("img");
            imageLink = imagePart[0]['src']
            titlePart = ddPart[0].find_all("p", attrs={"class": "search-title"})
            title = titlePart[0].find("a").get_text();
            starsPart = ddPart[0].find_all("span", attrs={"class": "score"})
            stars = starsPart[0]['title']
            stars = stars.rsplit(' ', 1)[1]
            pParts = ddPart[0].find_all("p")
            developerPart = pParts[1].find_all("a")
            developerName = developerPart[0].get_text()
            if first != 0:
                appIDList = appIDList + ","
            appIDList = appIDList + appID
            first = 1
            
            appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apkpure.com');
            insertIntoAppDetails(conn, appTableEntry)
        print("Done")
        firstCheck = 0
        if(len(names_table) == 0):
            checkMore = 0
    appMainTableEntry = (word, appIDList, 'null', 'apkpure.com');
#     print(appMainTableEntry)
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break